# Project Benson

## Exploring... Visualization

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels
import seaborn as sns
from numpy import linalg

import math
import patsy

from statsmodels.formula.api import ols

import folium
from folium import plugins

print(folium.__version__)

%matplotlib inline

0.5.0


In [5]:
!python -V

Python 3.6.3 :: Anaconda custom (64-bit)


In [6]:
print("Pandas version:",pd.__version__)
print("Numpy version:",np.__version__)

Pandas version: 0.20.3
Numpy version: 1.13.3


## Pick a week and play...

In [7]:
week = '170422' # Week of 2017, April 22
datafile = 'turnstile_%s.txt' % week
url = 'http://web.mta.info/developers/data/nyct/turnstile/%s' % datafile

# Specify location to store dataframes
mydir = '/home/joseph/ds/Projects/Project_Benson/Data'
#mydir =

df_pickle = '%s/turnstile_%s.pkl' % (mydir,week)
hourly_pickle = '%s/turnstile_%s_hourly.pkl' % (mydir,week)

## Data input

I start with the data off the MTA site.  I want a DateTime column immediately.

In [8]:
#df = pd.read_csv(url, parse_dates = [['DATE','TIME']])

# If we're just reading this in from the MTA site, we should do this correction now
# Comment this out otherwise
#df.columns.values[-1] = 'EXITS'


Or, if desired, read in previous work...

In [9]:
df = pd.read_pickle(df_pickle, compression='gzip')
hourly = pd.read_pickle(hourly_pickle, compression='gzip')

## Data Processing

In [10]:
df.head()

C/A LINENAME DIVISION     DESC  \
UNIT SCP      STATION DATE_TIME                                              
R051 02-00-00 59 ST   2017-04-15 00:00:00  A002  NQR456W      BMT  REGULAR   
                      2017-04-15 04:00:00  A002  NQR456W      BMT  REGULAR   
                      2017-04-15 08:00:00  A002  NQR456W      BMT  REGULAR   
                      2017-04-15 12:00:00  A002  NQR456W      BMT  REGULAR   
                      2017-04-15 16:00:00  A002  NQR456W      BMT  REGULAR   

                                           ENTRIES    EXITS  
UNIT SCP      STATION DATE_TIME                              
R051 02-00-00 59 ST   2017-04-15 00:00:00  6136580  2078941  
                      2017-04-15 04:00:00  6136613  2078947  
                      2017-04-15 08:00:00  6136639  2078987  
                      2017-04-15 12:00:00  6136799  2079109  
                      2017-04-15 16:00:00  6137076  2079173

In [11]:
weekly_aggregate = hourly.groupby('STATION').sum()
weekly_aggregate['TOTAL'] = weekly_aggregate['ENTRIES'] + weekly_aggregate['EXITS']
weekly_aggregate.reset_index(inplace=True)
weekly_aggregate.head()

,STATION,ENTRIES,EXITS,TOTAL
0,1 AV,130921.0,135974.0,266895.0
1,103 ST,196535.0,129877.0,326412.0
2,103 ST-CORONA,124827.0,89428.0,214255.0
3,104 ST,20499.0,5493.0,25992.0
4,110 ST,73940.0,56605.0,130545.0


In [12]:
geodata = pd.read_csv('../Data/Station_Geo.csv')

In [13]:
geodata.head()

,STATION,Latitude,Longitude
0,103 ST,40.796092,-73.961454
1,104 ST,40.695178,-73.844330
2,110 ST,40.795020,-73.944250
3,111 ST,40.697418,-73.836345
4,116 ST,40.805085,-73.954882


In [14]:
df2 = pd.merge(weekly_aggregate,geodata,on="STATION")

In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374 entries, 0 to 373
Data columns (total 6 columns):
STATION      374 non-null object
ENTRIES      374 non-null float64
EXITS        374 non-null float64
TOTAL        374 non-null float64
Latitude     374 non-null float64
Longitude    374 non-null float64
dtypes: float64(5), object(1)
memory usage: 20.5+ KB


In [16]:
stationArr = df2[['Latitude', 'Longitude','TOTAL']].as_matrix()

In [21]:
m = folium.Map([40.7528, -73.976522], zoom_start=12)

In [22]:
'''
for index, row in df2.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']],
                        radius=15,
                        popup=row['STATION'],
                        fill_color="#3db7e4", # divvy color
                       ).add_to(m)
m
'''

'\nfor index, row in df2.iterrows():\n    folium.CircleMarker([row[\'Latitude\'], row[\'Longitude\']],\n                        radius=15,\n                        popup=row[\'STATION\'],\n                        fill_color="#3db7e4", # divvy color\n                       ).add_to(m)\nm\n'

In [23]:
m.add_children(plugins.HeatMap(stationArr, radius=15))
m

/home/joseph/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  """Entry point for launching an IPython kernel.


In [30]:
df2[['STATION','Latitude', 'Longitude','TOTAL']].sort_values('TOTAL',ascending=False).head()

,STATION,Latitude,Longitude,TOTAL
61,34 ST-PENN STA,40.749719,-73.987823,1795211.0
232,GRD CNTRL-42 ST,40.751431,-73.976041,1641454.0
59,34 ST-HERALD SQ,40.749338,-73.987985,1467429.0
46,23 ST,40.742878,-73.992821,1242660.0
349,TIMES SQ-42 ST,40.754672,-73.986754,1223188.0


In [31]:
folium.Marker([40.749719, -73.987823],
              popup='34 ST-PENN STA'
             ).add_to(m)

In [35]:
folium.Marker([40.751431, -73.976041],
              popup='GRD CNTRL-42 ST',
              icon=folium.Icon(color='red',icon='1')
              ).add_to(m)


In [36]:
m

In [37]:
name_2017 = '2017_3mo'
hourly_2017_pickle = '%s/turnstile_%s_hourly.pkl' % (mydir,name_2017)
hourly_2017 = pd.read_pickle(hourly_2017_pickle, compression='gzip')



In [39]:
aggregate = hourly_2017.groupby('STATION').sum()
aggregate['TOTAL'] = aggregate['ENTRIES'] + aggregate['EXITS']
aggregate.reset_index(inplace=True)
aggregate.head()

,STATION,ENTRIES,EXITS,TOTAL
0,1 AV,1.812107e+06,1.894759e+06,3.706866e+06
1,103 ST,2.619699e+06,1.741515e+06,4.361214e+06
2,103 ST-CORONA,1.677419e+06,1.223505e+06,2.900924e+06
3,104 ST,2.776030e+05,8.843600e+04,3.660390e+05
4,110 ST,9.940580e+05,7.655420e+05,1.759600e+06


In [40]:
df3 = pd.merge(aggregate,geodata,on="STATION")

In [54]:
df3.sort_values('TOTAL',ascending=False).head(3)

,STATION,ENTRIES,EXITS,TOTAL,Latitude,Longitude
61,34 ST-PENN STA,1.325654e+07,1.133187e+07,2.458841e+07,40.749719,-73.987823
232,GRD CNTRL-42 ST,1.159893e+07,1.024549e+07,2.184443e+07,40.751431,-73.976041
59,34 ST-HERALD SQ,1.024411e+07,8.988547e+06,1.923266e+07,40.749338,-73.987985


In [57]:
m2 = folium.Map([40.7528, -73.976522], zoom_start=12)

for (index,row) in df3.sort_values('TOTAL',ascending=False).head(7).iterrows():
    folium.Marker([row['Latitude'], row['Longitude']],
              popup=row['STATION']
             ).add_to(m2)
m2

Hmm... while trying to get this ready for presentation... I'm suspicious about this geodata.  It's clearly good enough for a broad, general view.  But it looks odd for some of these.  Let's dig...

In [63]:
df3.sort_values('TOTAL',ascending=False).head(7)[['STATION','Latitude','Longitude']].as_matrix()

array([['34 ST-PENN STA', 40.749719, -73.98782299999999],
       ['GRD CNTRL-42 ST', 40.751431, -73.97604100000001],
       ['34 ST-HERALD SQ', 40.749338, -73.987985],
       ['23 ST', 40.742878000000005, -73.99282099999999],
       ['14 ST-UNION SQ', 40.734673, -73.989951],
       ['TIMES SQ-42 ST', 40.754672, -73.986754],
       ['42 ST-PORT AUTH', 40.756, -73.987]], dtype=object)

In [72]:
m3 = folium.Map([40.7528, -73.976522], zoom_start=12)


top7geo = [
    #['34 ST-PENN STA', 40.749719, -73.98782299999999],   # NOPE
    ['34 ST-PENN STA', 40.751, -73.991],   # OK
    #['GRD CNTRL-42 ST', 40.751431, -73.97604100000001],  # Could be better...
    ['GRD CNTRL-42 ST', 40.752283, -73.977519],  # OK
    ['34 ST-HERALD SQ', 40.749338, -73.987985],          # OK   
    ['23 ST', 40.742878000000005, -73.99282099999999],   # OK
    ['14 ST-UNION SQ', 40.734673, -73.989951],           # OK
    ['TIMES SQ-42 ST', 40.754672, -73.986754],           # OK
    #['42 ST-PORT AUTH', 40.756, -73.987]                 # Nah...
    ['42 ST-PORT AUTH', 40.75723755221195,-73.98980855941772]  # Sheesh... had to make it myself.
]

for index, (station,lat,lon) in enumerate(top7geo):
    folium.Marker([lat, lon],
          popup=station
         ).add_to(m3)
m3